<a href="https://colab.research.google.com/github/ErdemAslans/MultiDisease-Analysis-Platform/blob/main/Multidisease_Anaylsis_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install  segmentation_models_pytorch
!pip install monai
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00


In [ ]:
!pip install scikit-image

In [ ]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 54.2 MB/s eta 0:00:00


In [ ]:
!pip install bm3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.0/862.0 kB 21.6 MB/s eta 0:00:00


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import gradio as gr
import nibabel as nib
import numpy as np
import cv2
from skimage.transform import resize
from skimage import exposure
import pywt
from skimage.restoration import denoise_tv_chambolle
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

# ------------------ Model Tanımı ------------------ #
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAMBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channels, channels // reduction, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, kernel_size=1),
            nn.Sigmoid()
        )
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(2, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Kanal dikkat
        ca = self.channel_attention(x)
        x = x * ca
        # Uzamsal dikkat
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        sa = torch.cat([avg_out, max_out], dim=1)
        sa = self.spatial_attention(sa)
        x = x * sa
        return x

class MultiScaleFeatureFusion(nn.Module):
    def __init__(self, embed_dim, num_classes=1, dropout_rate=0.1):
        super(MultiScaleFeatureFusion, self).__init__()
        self.conv1 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(embed_dim, embed_dim // 2, kernel_size=3, padding=1)
        self.conv_pred = nn.Conv2d(embed_dim // 2, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)

        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        x1 = F.interpolate(self.conv1(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        x2 = self.conv2(x)
        x3 = F.interpolate(self.conv3(x), size=(x.shape[2], x.shape[3]), mode='bilinear', align_corners=False)
        fused = x1 + x2 + x3
        fused = self.dropout(fused)
        output = self.conv_pred(fused)
        return output

class ViTFeatureExtractor(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViTFeatureExtractor, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True, drop_rate=dropout_rate, drop_path_rate=0.2)
        self.vit.head = nn.Identity()
        self.embed_dim = self.vit.embed_dim

    def forward(self, x):
        x = self.vit.forward_features(x)
        return x

class SegFormerDecoderWithAttention(nn.Module):
    def __init__(self, embed_dim, num_classes=1, patch_size=16, dropout_rate=0.4):
        super(SegFormerDecoderWithAttention, self).__init__()
        self.patch_size = patch_size
        self.num_patches_side = 224 // self.patch_size
        self.linear_fuse = nn.Linear(embed_dim, embed_dim)
        self.conv_pred = nn.Conv2d(embed_dim, num_classes, kernel_size=1)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.cbam = CBAMBlock(embed_dim)
        self.multi_scale_fusion = MultiScaleFeatureFusion(embed_dim, num_classes, dropout_rate)
        nn.init.kaiming_normal_(self.linear_fuse.weight)
        nn.init.kaiming_normal_(self.conv_pred.weight)

    def forward(self, x):
        B, N, C = x.shape
        x = x[:, 1:, :]
        x = self.linear_fuse(x)
        x = x.transpose(1, 2).contiguous().view(B, C, self.num_patches_side, self.num_patches_side)
        x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=False)
        x = self.cbam(x)
        x = self.multi_scale_fusion(x)
        return x

class ViT_SegFormer_Model(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(ViT_SegFormer_Model, self).__init__()
        self.vit = ViTFeatureExtractor(dropout_rate)
        self.decoder = SegFormerDecoderWithAttention(embed_dim=self.vit.embed_dim, dropout_rate=dropout_rate)

    def forward(self, x):
        features = self.vit(x)
        output = self.decoder(features)
        return output

# Cihaz seçimi (GPU veya CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli yükleme
model = ViT_SegFormer_Model(dropout_rate=0.2).to(device)
weight_path = "/content/drive/MyDrive/Tümör/best_model.pth"  # Kendi ağırlık dosyanızın yolunu ekleyin
state_dict = torch.load(weight_path, map_location=device)
model.load_state_dict(state_dict)
model.eval()

# ------------------ Ön İşleme Fonksiyonları ------------------ #
def enhance_contrast(image, clip_limit=0.6, tile_grid_size=(32,32), gamma=0.3):
    min_val = np.min(image)
    max_val = np.max(image)
    if max_val - min_val == 0:
        image = np.zeros_like(image)
    else:
        image = (image - min_val) / (max_val - min_val)  # [0, 1]

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    image = clahe.apply((image * 255).astype(np.uint8)) / 255.0

    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
    image = cv2.LUT((image * 255).astype(np.uint8), table).astype(np.float32) / 255

    laplacian = cv2.Laplacian(image.astype(np.float64), cv2.CV_64F)
    laplacian = np.clip(laplacian, 0, 1)
    laplacian_weight = 0.33
    enhanced_image = image + laplacian_weight * laplacian
    enhanced_image = np.clip(enhanced_image, 0, 1)
    return enhanced_image

def apply_nlm_denoising(image):
    denoised_image = cv2.fastNlMeansDenoising(np.uint8(image * 255), None, h=10, templateWindowSize=7, searchWindowSize=21)
    denoised_image = denoised_image.astype(np.float32) / 255.0
    return denoised_image

def apply_wavelet_denoising(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    coeffs = list(coeffs)
    coeffs[1:] = [tuple(pywt.threshold(c, value=0.1, mode='soft') for c in level) for level in coeffs[1:]]
    denoised_image = pywt.waverec2(coeffs, 'haar')
    if denoised_image.shape != image.shape:
        denoised_image = resize(denoised_image, image.shape, mode='reflect', anti_aliasing=True)
    return denoised_image

def apply_total_variation_denoising(image):
    denoised_image = denoise_tv_chambolle(image, weight=0.1)
    return denoised_image

def process_single_nii(image_nii, mask_nii, view, slice_idx, target_size=(224,224)):
    img = image_nii.get_fdata()
    if mask_nii is not None:
        mask = mask_nii.get_fdata()
    else:
        mask = np.zeros_like(img)

    # Görüntü dilimleme
    if view == 'sagittal':
        max_slices = img.shape[0]
        img_slice = img[slice_idx, :, :] if slice_idx < max_slices else img[max_slices//2,:,:]
        mask_slice = mask[slice_idx, :, :] if slice_idx < max_slices else mask[max_slices//2,:,:]
    elif view == 'coronal':
        max_slices = img.shape[1]
        img_slice = img[:, slice_idx, :] if slice_idx < max_slices else img[:, max_slices//2,:]
        mask_slice = mask[:, slice_idx, :] if slice_idx < max_slices else mask[:, max_slices//2,:]
    elif view == 'axial':
        max_slices = img.shape[2]
        img_slice = img[:, :, slice_idx] if slice_idx < max_slices else img[:,:,max_slices//2]
        mask_slice = mask[:, :, slice_idx] if slice_idx < max_slices else mask[:,:,max_slices//2]

    # Resize
    img_slice = resize(img_slice, target_size, mode='reflect', anti_aliasing=True)
    mask_slice = resize(
        mask_slice, target_size, mode='reflect', anti_aliasing=False,
        order=0, preserve_range=True
    )

    # İşlemler
    img_slice = enhance_contrast(img_slice)
    img_slice = apply_nlm_denoising(img_slice)
    img_slice = apply_wavelet_denoising(img_slice)
    img_slice = apply_total_variation_denoising(img_slice)

    # Maskeyi binarize et
    mask_slice = (mask_slice > 0.05).astype(np.float32)

    # Maskeye göre parlaklık ayarlama
    highlight_factor = 1.8
    darken_factor = 0.8
    img_slice_masked = img_slice * mask_slice * highlight_factor
    img_slice_non_masked = img_slice * (1 - mask_slice) * darken_factor
    img_slice = img_slice_masked + img_slice_non_masked
    img_slice = np.clip(img_slice, 0, 1).astype(np.float32)

    return img_slice, mask_slice

def gradio_process(image_file, mask_file, view, slice_idx):
    # NIfTI dosyalarını oku
    image_nii = nib.load(image_file.name)
    mask_nii = nib.load(mask_file.name) if mask_file is not None else None

    # İşlenmiş görüntü ve maske
    img_slice, mask_slice = process_single_nii(image_nii, mask_nii, view, slice_idx, target_size=(224,224))

    # Modelin girdi formatına uygun dönüştürme
    # Model 3 kanallı girdi beklediği varsayılarak tek kanallı görüntüyü 3 kanal haline getiriyoruz:
    img_tensor = torch.from_numpy(img_slice).unsqueeze(0).unsqueeze(0)  # (1, 1, H, W)
    img_tensor = img_tensor.repeat(1, 3, 1, 1) # (1, 3, 224, 224)
    img_tensor = img_tensor.to(device)

    # Model tahmini
    with torch.no_grad():
        logits = model(img_tensor)    # (1, 1, 224, 224)
        pred_prob = torch.sigmoid(logits)
        pred_mask = (pred_prob > 0.5).float()

    pred_mask_np = pred_mask.cpu().squeeze().numpy()  # (224, 224) boyutunda

    # Görüntü ve modelin tahmini maskeyi döndür
    return (img_slice, pred_mask_np)

with gr.Blocks() as demo:
    gr.Markdown("## Tek NIfTI Dosyası Üzerinde Ön İşleme, Model ile Segmentasyon ve Görselleştirme")
    with gr.Row():
        image_input = gr.File(label="NIfTI Görüntü Dosyası (.nii veya .nii.gz)")
        mask_input = gr.File(label="Maske Dosyası (Opsiyonel)")
    view_choice = gr.Dropdown(choices=["sagittal", "coronal", "axial"], value="axial", label="Görüntüleme Ekseni")
    slice_choice = gr.Slider(minimum=0, maximum=200, step=1, value=50, label="Dilim Numarası (seçilen eksene göre)")

    run_button = gr.Button("İşle ve Göster")

    output_image = gr.Image(label="İşlenmiş Görüntü")
    output_mask = gr.Image(label="Model Tahmin Maskesi")

    def on_run(image_file, mask_file, view, slice_idx):
        img_slice, pred_mask = gradio_process(image_file, mask_file, view, slice_idx)
        return img_slice, pred_mask

    run_button.click(fn=on_run, inputs=[image_input, mask_input, view_choice, slice_choice], outputs=[output_image, output_mask])

demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

<ipython-input-7-f8311998da98>:117: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5911d3788e3d3fcb04.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8